In [0]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=cec08950c7c0d018689c450a2340aaa8d3f6c81fd9d2cdeef9069bfab8d3899f
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS 508/Class/Week3/IF_Train.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS 508/Class/Week3/IF_Test.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(2999, 32)
  MONTH  WEEKOFMONTH  DAYOFWEEK  ...  YEAR  BASEPOLICY FRAUDFOUND
0   Jul            3     Sunday  ...  1994   Collision        Yes
1   Nov            5     Monday  ...  1994  All_Perils        Yes
2   Jan            1     Monday  ...  1994  All_Perils        Yes
3   Dec            1     Monday  ...  1994  All_Perils        Yes
4   Dec            5  Wednesday  ...  1994   Collision        Yes

[5 rows x 32 columns]
(12918, 32)
  MONTH  WEEKOFMONTH 

In [0]:
categoricalFeatures = ["MONTH",
"DAYOFWEEK",
"MAKE",
"ACCIDENTAREA",
"DAYOFWEEKCLAIMED",
"MONTHCLAIMED",
"SEX",
"MARITALSTATUS",
"FAULT",
"POLICYTYPE",
"VEHICLECATEGORY",
"VEHICLEPRICE",
"DAYS_POLICY_ACCIDENT",
"DAYS_POLICY_CLAIM",
"PASTNUMBEROFCLAIMS",
"AGEOFVEHICLE",
"AGEOFPOLICYHOLDER",
"POLICEREPORTFILED",
"WITNESSPRESENT",
"AGENTTYPE",
"NUMBEROFSUPPLIMENTS",
"ADDRESSCHANGE_CLAIM",
"NUMBEROFCARS",
"YEAR",
"BASEPOLICY"]


#Copy Train data excluding target
trainData_Copy = train_data.iloc[:, :-1].copy()
testData_Copy = test_data.iloc[:, :-1].copy()

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)
#print(combined_Data['FRAUDFOUND'])

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
y_train=train_data["FRAUDFOUND"]
y_test=test_data["FRAUDFOUND"]

train_data["FRAUDFOUND"]=train_data["FRAUDFOUND"].map({"Yes":1,"No":0})
test_data["FRAUDFOUND"]=test_data["FRAUDFOUND"].map({"Yes":1,"No":0})

#Select just Target Column
y_train = train_data.iloc[:, -1]
y_test = test_data.iloc[:, -1]

print(X_train.shape)
print(X_test.head()) 

print(y_train.shape)
print(y_test.head()) 

(2999, 146)
   WEEKOFMONTH  WEEKOFMONTHCLAIMED  ...  BASEPOLICY_Collision  BASEPOLICY_Liability
0            3                   4  ...                     1                     0
1            5                   1  ...                     0                     0
2            1                   1  ...                     0                     0
3            1                   3  ...                     0                     0
4            5                   1  ...                     1                     0

[5 rows x 146 columns]
(2999,)
0    1
1    1
2    1
3    1
4    1
Name: FRAUDFOUND, dtype: int64


In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))


accuracy Score (training) for Decision TreE:0.884038
Confusion Matrix for Decision Tree
[[10975  1445]
 [   53   445]]


In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


{'min_samples_split': 20, 'max_depth': 3}
accuracy Score (training) after hypertuning for Decision Tree:0.851138
Confusion Matrix after hypertuning for Decision Tree
[[10802  1618]
 [  305   193]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.97      0.87      0.92     12420
           1       0.11      0.39      0.17       498

    accuracy                           0.85     12918
   macro avg       0.54      0.63      0.54     12918
weighted avg       0.94      0.85      0.89     12918

=== All AUC Scores ===
[0.89740385 0.9525     0.8725     0.86769231 0.81740385 0.73254808
 0.71913462 0.79019231 0.79144231 0.80394477]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.8244762080867851


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))




accuracy Score (training) for RandomForest:0.945889
Confusion Matrix for Random Forest:
[[11828   592]
 [  107   391]]


In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 50, 'min_samples_split': 10, 'max_depth': 17}
accuracy Score (training) after hypertuning for Random Forest:0.935903
Confusion Matrix after hypertuning for Random Forest:
[[11882   538]
 [  290   208]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.98      0.96      0.97     12420
           1       0.28      0.42      0.33       498

    accuracy                           0.94     12918
   macro avg       0.63      0.69      0.65     12918
weighted avg       0.95      0.94      0.94     12918

=== All AUC Scores ===
[0.89269231 0.91730769 0.92682692 0.8975     0.83875    0.7425
 0.74990385 0.81384615 0.82       0.84053254]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8439859467455622


In [0]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))

#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

accuracy Score (training) for Boosting:0.909661
Confusion Matrix for boosting:
[[11546   874]
 [  293   205]]
{'n_estimators': 5, 'learning_rate': 0.01}


In [0]:
#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))

#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())


accuracy Score (training) after hypertuning for Boosting:0.961449
Confusion Matrix after hypertuning for Boosting:
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     12420
           1       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weighted avg       0.92      0.96      0.94     12918

=== All AUC Scores ===
[0.89740385 0.9525     0.8725     0.86769231 0.81740385 0.73254808
 0.71913462 0.79019231 0.79144231 0.80394477]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.8244762080867851


In [0]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 2600, 1: 399})
Resampled dataset shape Counter({0: 2600, 1: 1300})


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.87466667]
    fold  1:  [0.88000000]
    fold  2:  [0.89733333]
    fold  3:  [0.90654206]
    ----
    MEAN:     [0.88963551] + [0.01286465]
    FULL:     [0.88962988]

model  1:     [RandomForestClassifier]
    fold  0:  [0.88400000]
    fold  1:  [0.88266667]
    fold  2:  [0.88400000]
    fold  3:  [0.88651535]
    ----
    MEAN:     [0.88429551] + [0.00139243]
    FULL:     [0.88429476]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.82133333]
    fold  1:  [0.85600000]
    fold  2:  [0.84533333]
    fold  3:  [0.84112150]
    ----
    MEAN:     [0.84094704] + [0.01255547]
    FULL:     [0.84094698]



In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_train)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(y_test,y_pred))
print("=== Classification Report ===")
print(classification_report(y_test,y_pred))


Final prediction score for ensemble methods: [0.93613562]
Confusion Matrix after STACKING for Boosting:
[[11905   515]
 [  310   188]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.97      0.96      0.97     12420
           1       0.27      0.38      0.31       498

    accuracy                           0.94     12918
   macro avg       0.62      0.67      0.64     12918
weighted avg       0.95      0.94      0.94     12918



In [0]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()


,0,1
0,0.181836,0.818164
1,0.181836,0.818164
2,0.181836,0.818164
3,0.181836,0.818164
4,0.500009,0.499991
